In [16]:
import pickle

with open('best_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

In [17]:
with open('scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

In [18]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern

# Define GLCM parameters
glcm_distances = [1, 2, 3, 4]
glcm_angles = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2]
glcm_properties = ['contrast', 'homogeneity', 'energy', 'correlation', 'dissimilarity', 'ASM']
hsv_properties = ['hue', 'saturation', 'value']

# Function to extract features from a single image
def extract_features(image_path, image_size=(256, 256)):
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)

    # Convert the image to HSV and grayscale
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # --- Extract HSV Features (mean and std) ---
    hsv_features = []
    for property_name in hsv_properties:
        property_value = hsv_image[:, :, hsv_properties.index(property_name)].ravel()
        hsv_features.extend([np.mean(property_value), np.std(property_value)])

    # --- Extract GLCM Features ---
    glcm = graycomatrix(gray_image, distances=glcm_distances, angles=glcm_angles, symmetric=True, normed=True)
    glcm_features = []
    for property_name in glcm_properties:
        property_value = graycoprops(glcm, property_name).ravel()
        glcm_features.extend([np.mean(property_value), np.std(property_value)])

    # --- Extract Color Histogram Features ---
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    color_histogram_features = cv2.normalize(hist, hist).flatten()

    # --- Extract LBP Features ---
    lbp = local_binary_pattern(gray_image, 24, 3, method="uniform")
    (lbp_hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 24 + 3), range=(0, 24 + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-7)  # Normalize the histogram
    lbp_features = lbp_hist

    # --- Extract Sobel Edge Features ---
    sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    sobelx_mean, sobelx_std = np.mean(sobelx), np.std(sobelx)
    sobely_mean, sobely_std = np.mean(sobely), np.std(sobely)
    sobel_features = [sobelx_mean, sobelx_std, sobely_mean, sobely_std]

    # Combine all features
    combined_features = np.concatenate([hsv_features, glcm_features, color_histogram_features, lbp_features, sobel_features])
    
    return combined_features


In [22]:
import os

# Define the class names
class_names = ['ganas', 'jinak', 'normal']


def predict_image(image_path):
    # Extract features from the image
    features = extract_features(image_path).reshape(1, -1)
    
    # Scale the features using the loaded scaler
    features_scaled = scaler.transform(features)
    
    # Predict using the loaded model
    predicted_class_index = loaded_model.predict(features_scaled)[0]
    predicted_class = class_names[predicted_class_index]
    
    return predicted_class


# List of test images
test_images = ['data_test/ganas.jpg', 'data_test/ganas_1.jpg', 'data_test/jinak.jpg', 'data_test/jinak_1.jpg', 'data_test/normal.jpg', 'data_test/normal_1.jpg']

# Predict and print the results
for image_path in test_images:
    prediction = predict_image(image_path)
    print(f"The predicted class for {os.path.basename(image_path)} is: {prediction}")


The predicted class for ganas.jpg is: ganas
The predicted class for ganas_1.jpg is: ganas
The predicted class for jinak.jpg is: jinak
The predicted class for jinak_1.jpg is: jinak
The predicted class for normal.jpg is: normal
The predicted class for normal_1.jpg is: normal
